In [20]:
import pandas as pd
import numpy as np
from datetime import datetime
import plotly.graph_objects as go

In [21]:
data = pd.read_csv("../data/salary.csv")

# Cleaning 

This code cleans data by: 
- removing unnecessary columns
- calculating total expected salary
- filling missing values with zero
- standardizing department names 
- computes hourly rates 
- overtime rates
- total weekly hours worked 
- converts the current years data to real time budget tracking. 

In [22]:
# drops cols we dont need 
cols_to_drop = ['jobTitle', 'Other', 'ObjectId']
data = data.drop(columns=cols_to_drop)
# calculates total expected salary 
data['Salary_Total'] = data['Annual_Rate'] + data['Incentive_Allowance']
# Filling missing data with 0 
data = data.fillna(0)
# merging the departments together 
data['Department'] = data['Department'].replace('Louisville Metro Police', 'Louisville Metro Police Department')
data['Department'] = data['Department'].replace('Department of Corrections', 'Metro Corrections')
# calculates total weekly hrs worked 
data['Hr_Rate'] = data['Regular_Rate'] / 2080
data['Ot_Rate'] = data['Hr_Rate'] * 1.5
data['Hr_Worked'] = data['Overtime_Rate'] / data['Ot_Rate'] / 52 + 40
data.replace([np.inf, -np.inf], 40, inplace=True)

'''fix this soon
add update_check.py to this file to have the dynamic var

date_div = "November 27, 2024"
'''

# Define the date
date_div = "November 27, 2024"

# Convert the string to a datetime object
date_object = datetime.strptime(date_div, "%B %d, %Y")

# Get the week number of the year
week = date_object.isocalendar()[1]

# Convert the string to a datetime object
date_object = datetime.strptime(date_div, "%B %d, %Y")

# Get the year
year = date_object.year


# Check if there are any rows where the CalYear is equal to the specified year
if (data['CalYear'] == year).any():
    # Convert Annual_Rate to weekly rate by dividing by 52
    data.loc[data['CalYear'] == year, 'Annual_Rate'] = data.loc[data['CalYear'] == year, 'Annual_Rate'] / 52

    # Scale the weekly rate for the specific week
    data.loc[data['CalYear'] == year, 'Annual_Rate'] = data.loc[data['CalYear'] == year, 'Annual_Rate'] * week


In [23]:
# # Check if there are any rows where the CalYear is equal to the specified year
# if (data['CalYear'] == year).any():
#     # Convert Annual_Rate to weekly rate by dividing by 52
#     data.loc[data['CalYear'] == year, 'Annual_Rate'] = data.loc[data['CalYear'] == year, 'Annual_Rate'] / 52

#     # Scale the weekly rate for the specific week
#     data.loc[data['CalYear'] == year, 'Annual_Rate'] = data.loc[data['CalYear'] == year, 'Annual_Rate'] * week


<pre>
+---------------------------------------------------------------------------------------------------------+
|+-------------------------------------------------------------------------------------------------------+|
||[1. logo] 2. Louisville Metro Government Salary tracker   [3. Department drop down][ 4. year drop down]|| 
|+-------------------------------------------------------------------------------------------------------+|
|+--------+     6. salary spend 8. actual salary spend +-----+     +-------------------------------------+|
||5. plot |     7. salary data  9. salary data         |12. %|     |  11. horizontal bar plot department || 
||        |                                            +-----+     |                                     || 
|+--------+                                                        |                                     || 
|                                                                  |                                     || 
|+---------------------------------------------------------------+ |                                     || 
|| 10. Top employees salary deviation data frame                 | |                                     || 
||                                                               | |                                     || 
||                                                               | |                                     || 
||                                                               | |                                     || 
||                                                               | |                                     || 
||                                                               | |                                     || 
||                                                               | |                                     || 
|+---------------------------------------------------------------+ +-------------------------------------+|
+---------------------------------------------------------------------------------------------------------+
</pre>


## code for 5, 7, 9, and 12 

- 5 Will be a gauge plot?
- 7 will be the calculation of total salary spend up till today or historical total
- 9 actual salary spend of total salary spend up till today or historical total
- 12 will be the percent difference in the salary spend 

In [24]:
# gauge plot
# 2019-2024
gauge = data.groupby(['CalYear'])[['YTD_Total', 'Salary_Total']].sum().reset_index()
gauge.head(10)

,CalYear,YTD_Total,Salary_Total
0,2019,3.278264e+08,3.419880e+08
1,2020,3.331710e+08,3.396643e+08
2,2021,3.307436e+08,3.512589e+08
3,2022,3.901357e+08,3.702980e+08
4,2023,3.858792e+08,4.272285e+08
5,2024,3.730627e+08,4.506993e+08


In [25]:
def plot_info(year, gauge):
    # Filter the data for the given year
    year_filter = gauge[gauge['CalYear'] == year]
    
    # Extract actual and expected values
    actual = year_filter['YTD_Total'].iloc[0]
    expected = year_filter['Salary_Total'].iloc[0]

    # Create the gauge plot
    steps = [
        {'range': [0, expected], 'color': '#004080'}  # Dark blue for expected salary
    ]
    
    # Add yellow step if actual exceeds expected
    if actual > expected:
        steps.append({'range': [expected, actual], 'color': 'yellow'})  # Yellow for actual salary
    
    fig = go.Figure(go.Indicator(
        mode="gauge+number+delta",
        value=actual,
        delta={
            'reference': expected,
            'increasing': {'color': "red"},
            'decreasing': {'color': "green"}
        },
        gauge={
            'axis': {'range': [0, expected * 1.1]},  
            'bar': {'color': 'rgba(0,0,0,0)'},
            'steps': steps,
            'threshold': {
                'line': {'color': "red", 'width': 4},  
                'thickness': .95,
                'value': actual  # Place the red line at the actual
            }
        },
        title={'text': f"{year} Salary Spend vs. Budget", 'font': {'size': 20}},
    ))

    fig.update_layout(
        title_font={'size': 20},  # Set font size for the overall layout title
    )

    # Show the figure
    fig.show()


# Example usage:
plot_info(2024, gauge)


In [26]:
# Example usage:
plot_info(2019, gauge)

# Whole department views

In [27]:
department = data.groupby(['CalYear', 'Department'])[['YTD_Total', 'Salary_Total']].sum().reset_index()

In [28]:
department.head()

,CalYear,Department,YTD_Total,Salary_Total
0,2019,Air Pollution Control,3161000.01,3357059.36
1,2019,Belle of Louisville,1288272.26,2255142.24
2,2019,Commonwealth Attorney,1076553.31,1254064.50
3,2019,Coroner,883494.80,1006419.20
4,2019,County Attorney,5253504.81,6583707.28


# Individual Department Views

<pre>
+---------------------------------------------------------------------------------------------------------+
|+-------------------------------------------------------------------------------------------------------+|
||[1. logo] 2. Louisville Metro Government Salary tracker   [3. Department drop down][ 4. year drop down]|| 
|+-------------------------------------------------------------------------------------------------------+|
|+--------+     6. salary spend 8. actual salary spend +-----+     +-------------------------------------+|
||5. plot |     7. salary data  9. salary data         |12. %|     |  11. horizontal bar plot department || 
||        |                                            +-----+     |                                     || 
|+--------+                                                        |                                     || 
|                                                                  |                                     || 
|+---------------------------------------------------------------+ |                                     || 
|| 10. Top employees salary deviation data frame                 | |                                     || 
||                                                               | |                                     || 
||                                                               | |                                     || 
||                                                               | |                                     || 
||                                                               | |                                     || 
||                                                               | |                                     || 
||                                                               | |                                     || 
|+---------------------------------------------------------------+ +-------------------------------------+|
+---------------------------------------------------------------------------------------------------------+
</pre>


In [29]:
lmpd = department[department['Department'] == 'Louisville Metro Police Department'].copy()
lmpd = lmpd[lmpd['CalYear'] != 2024].copy()
lmpd

,CalYear,Department,YTD_Total,Salary_Total
19,2019,Louisville Metro Police Department,1.022160e+08,9.862518e+07
56,2020,Louisville Metro Police Department,1.061527e+08,1.040606e+08
93,2021,Louisville Metro Police Department,1.033841e+08,1.020937e+08
130,2022,Louisville Metro Police Department,1.224085e+08,1.051902e+08
175,2023,Louisville Metro Police Department,1.223053e+08,1.182766e+08


In [30]:
lmpd.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5 entries, 19 to 175
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   CalYear       5 non-null      int64  
 1   Department    5 non-null      object 
 2   YTD_Total     5 non-null      float64
 3   Salary_Total  5 non-null      float64
dtypes: float64(2), int64(1), object(1)
memory usage: 200.0+ bytes


In [31]:
lmpd['Discrepancy'] = lmpd['YTD_Total'] - lmpd['Salary_Total']
lmpd['Discrepancy_Ratio'] = lmpd['Discrepancy'] / lmpd['Salary_Total']

lmpd

,CalYear,Department,YTD_Total,Salary_Total,Discrepancy,Discrepancy_Ratio
19,2019,Louisville Metro Police Department,1.022160e+08,9.862518e+07,3590832.44,0.036409
56,2020,Louisville Metro Police Department,1.061527e+08,1.040606e+08,2092070.71,0.020104
93,2021,Louisville Metro Police Department,1.033841e+08,1.020937e+08,1290365.61,0.012639
130,2022,Louisville Metro Police Department,1.224085e+08,1.051902e+08,17218319.86,0.163687
175,2023,Louisville Metro Police Department,1.223053e+08,1.182766e+08,4028742.06,0.034062


In business, a discrepancy ratio between expected salary spend (Salary_Total) and actual salary spend (YTD_Total) typically falls within 0 to 5% as an acceptable range. A discrepancy within this range usually indicates minor variations in salary spending, such as unexpected overtime, temporary staffing, or adjustments to salary levels, which are often manageable within the overall budget. A discrepancy ratio of more than 10% would generally raise concerns, signaling potential overspending on salaries, and would require a deeper investigation into the causes, such as unplanned hiring, excessive overtime, or compensation adjustments.

For the Louisville Metro Police Department, the discrepancy ratios from 2019 to 2023 show a range from 0.01 to 0.16. The ratios in 2019, 2020, 2021, and 2023 are all within a 0.01 to 0.04 range, which suggests a relatively small and manageable variance in salary spending. However, the discrepancy ratio for 2022 is significantly higher at 0.16, indicating a 16% overspend on salaries compared to the planned budget for that year. This could point to specific factors in 2022 that led to a sharp increase in salary expenses, such as a one-time hiring initiative, increased overtime costs, or other unforeseen budgetary pressures.

# LMPD indv level

In [32]:
indv = pd.read_csv('data/SalaryData.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'data/SalaryData.csv'

In [ ]:
cols_to_drop = ['Regular_Rate', 'Overtime_Rate', 'Other', 'ObjectId']
indv = indv.drop(columns=cols_to_drop)
indv['Salary_Total'] = indv['Annual_Rate'] + indv['Incentive_Allowance']
final_drop = ['Annual_Rate', 'Incentive_Allowance']
indv = indv.drop(columns=final_drop)
indv = indv.fillna(0)
indv['Department'] = indv['Department'].replace('Louisville Metro Police', 'Louisville Metro Police Department')
indv = indv[indv['CalYear'] != 2024].copy()

In [ ]:
indv = indv[indv['Department'] == 'Louisville Metro Police Department'].copy()
# indv.head(2)

In [ ]:
indv['Discrepancy'] = indv['YTD_Total'] - indv['Salary_Total']
indv['Discrepancy_Ratio'] = indv['Discrepancy'] / indv['Salary_Total']


In [ ]:
indv.head(2)

Top 10 Employees with the Highest Discrepancies

In [ ]:
def top_10_discrepancy(indv, enter_year):
    # Filter the DataFrame for the given year
    indv_year = indv[indv['CalYear'] == enter_year]
    
    # Sort by 'Discrepancy' in descending order to get the top 10 employees
    top_10_employees = indv_year.sort_values(by='Discrepancy_Ratio', ascending=False).head(10)
    
    # Return the top 10 employees with their discrepancies
    return top_10_employees[['CalYear', 'Employee_Name', 'jobTitle', 'Discrepancy', 'Discrepancy_Ratio', 'YTD_Total', 'Salary_Total']]

In [ ]:
# Example usage:
top_emp_dis_2019 = top_10_discrepancy(indv, 2021)
top_emp_dis_2019

In [ ]:
top_emp_dis_2020 = top_10_discrepancy(indv, 2020)
top_emp_dis_2020

In [ ]:
top_emp_dis_2022 = top_10_discrepancy(indv, 2022)
top_emp_dis_2022

Job Title with the Largest Individual Discrepancy by Year:

In [ ]:
# Group by CalYear and jobTitle to find the row with the largest discrepancy per year
largest_discrepancy_by_year = indv.loc[indv.groupby('CalYear')['Discrepancy'].idxmax()]

# Display the job title and the corresponding discrepancy
largest_discrepancy_by_year[['CalYear', 'jobTitle', 'Discrepancy']]

top 10 job titles by avg discrepancy by year

In [ ]:
def top_10_job_titles_by_avg_discrepancy(indv, enter_year):
    # Filter the DataFrame for the given year
    indv_year = indv[indv['CalYear'] == enter_year]
    
    # Group by 'jobTitle' and calculate the average 'Discrepancy' for each job title
    avg_discrepancy_by_job = indv_year.groupby('jobTitle')['Discrepancy'].mean()
    
    # Sort by the average 'Discrepancy' in descending order to get the top 10 job titles
    top_10_job_titles = avg_discrepancy_by_job.sort_values(ascending=False).head(10)
    
    # Return the relevant columns (jobTitle and average Discrepancy)
    return top_10_job_titles.reset_index()[['jobTitle', 'Discrepancy']]

In [ ]:
# Example usage:
top_10_job_titles_by_avg_discrepancy_2023 = top_10_job_titles_by_avg_discrepancy(indv, 2023)
top_10_job_titles_by_avg_discrepancy_2023

Average Discrepancy per Job Title:

In [ ]:
def avg_discrepancy_by_job(indv):
    # Group by 'jobTitle' and calculate the average discrepancy (YTD_Total - Salary_Total)
    avg_discrepancy_per_job = indv.groupby('jobTitle')['Discrepancy'].mean()
    
    # Sort the result by average discrepancy in descending order
    avg_discrepancy_per_job_sorted = avg_discrepancy_per_job.sort_values(ascending=False)
    
    # Return the sorted average discrepancies
    return avg_discrepancy_per_job_sorted

In [ ]:
# Example usage:
avg_discrepancy = avg_discrepancy_by_job(indv)
avg_discrepancy